In [1]:
!pip install mnist

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
!cp "/content/drive/My Drive/imimic/wann/mnist_test_c_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_test_cn_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_test_features.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_test_hu_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_test_radiomics.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_c_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_cn_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_features.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_hu_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_radiomics.npy" .

In [4]:
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import mnist
random.seed(1)
np.random.seed(1)
from __future__ import print_function
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K


Using TensorFlow backend.


In [0]:
PREFIX = 'features'

num_classes = 10

x_train = np.load(f'mnist_train_{PREFIX}.npy')
y_train = mnist.train_labels()
x_test = np.load(f'mnist_test_{PREFIX}.npy')
y_test = mnist.test_labels()


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [7]:
from keras.callbacks import ModelCheckpoint

batch_size = 128
epochs = 1000

filepath = 'weights-{epoch:02d}-{acc:.4f}-{val_acc:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# fits the model on batches with real-time data augmentation:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/1000





60000/60000 [==============================] - 4s 68us/step - loss: 3.1434 - acc: 0.3674 - val_loss: 1.4135 - val_acc: 0.5100

Epoch 00001: val_acc improved from -inf to 0.51000, saving model to weights-01-0.3674-0.5100.hdf5
Epoch 2/1000
60000/60000 [==============================] - 3s 51us/step - loss: 1.3904 - acc: 0.4959 - val_loss: 1.2259 - val_acc: 0.5886

Epoch 00002: val_acc improved from 0.51000 to 0.58860, saving model to weights-02-0.4959-0.5886.hdf5
Epoch 3/1000
60000/60000 [==============================] - 3s 54us/step - loss: 1.2771 - acc: 0.5449 - val_loss: 1.1457 - val_acc: 0.6175

Epoch 00003: val_acc improved from 0.58860 to 0.61750, saving model to weights-03-0.5449-0.6175.hdf5
Epoch 4/1

KeyboardInterrupt: ignored

In [0]:
!tar cf weights_features.tar *.hdf5
!xz weights_features.tar
!cp weights_features.tar.xz "/content/drive/My Drive/imimic/wann"